<a href="https://colab.research.google.com/github/claudioalvesmonteiro/machinelearning-CNPJ/blob/master/pre-processamento/FEATURE_ENGINEERING_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

   # Projeto de Análise de Cŕedito CESAR School
   
   ## Feature Engineering
   
   Claudio Alves Monteiro
   
   Marcos Antonio Almeida Souto Júnior
   
   Virgínia Heimann
   
   Kayo Renato da Silva Nascimento
   
   Rosely Cabral

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://mirror.nbtelecom.com.br/apache/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
!tar xf spark-2.4.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
# import modules
import os
import pandas as pd
from pyspark.sql import functions as SF
import pyspark.sql.types as ST

In [0]:
# funcao para contagem de valores categoricos
def countGenTab(df, column):
    # groupby, count and sort pyspark
    tab = df.select(column).groupby(column).count().alias('count').sort('count', ascending=False)
    # transform to pandas df
    tab = tab.toPandas()
    # calculate proportion
    tab['prop'] = round(tab['count']/sum(tab['count'])*100, 2)
    return tab

## Tratamento

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# import data
path = 'drive/My Drive/Colab Notebooks/ProjetoCESARSchool/codes/data/'

df = spark.read.csv(path+'output5.csv',
                    sep=',',
                    encoding='utf-8',
                    header=True,
                    inferSchema=False)

### Contagem e drop de nulos

In [0]:
df.count()

64912

In [0]:
from pyspark.sql.functions import isnan, when, count, col


df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()


df = df.na.drop()

+---+----+--------+--------+-------+----------+-----------------+-----+--------------+----+--------+-------------------+---+---------+------------+----------+---------+------------+--------------+-------------------------+----------------------+----------------------------+--------------------+----+----------+------------------+--------------+-----------------------+--------------+---------+-----------------+
|_c0|CNPJ|REF_DATE|DATA_REF|CS_ALVO|QTD_SOCIOS|NATUREZA_JURIDICA|IDADE|CAPITAL_SOCIAL|TIPO|SITUACAO|ATIVIDADE_PRINCIPAL| UF|MUNICIPIO|MUNICIPIO_UF|code_muni2|code_muni|dist_capital|area_municipio|taxa_atividade_18anosmais|porcent_pop_saneamento|porcent_mulheres10a17_filhos|mortalidade_infantil|IDHM|IDHM_renda|taxa_analfabetismo|porcent_pobres|municipio_metropolitano|municipio_nome|populacao|municipio_nome_uf|
+---+----+--------+--------+-------+----------+-----------------+-----+--------------+----+--------+-------------------+---+---------+------------+----------+---------+------

In [0]:
df.count()

64912

In [0]:
df.where(SF.col('populacao') != 'NaN').count()


64912

In [0]:
# NORMALIZAÇÃO DAS VARIÁVEIS NUMÉRICAS


cols = ['IDADE','CAPITAL_SOCIAL','taxa_atividade_18anosmais',
       'porcent_pop_saneamento','dist_capital',
       'area_municipio','taxa_atividade_18anosmais',
       'porcent_pop_saneamento','porcent_mulheres10a17_filhos',
       'mortalidade_infantil','IDHM',
       'IDHM_renda','taxa_analfabetismo',
        'porcent_pobres','populacao']

for col_name in cols:
    df = df.withColumn(col_name, SF.col(col_name).cast('double'))
    


In [0]:


from sklearn.preprocessing import StandardScaler

df_pandas = df.toPandas()

scaled_features = df_pandas.copy()
col_names = cols
features = scaled_features[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features[col_names] = features
#print(scaled_features)

df = spark.createDataFrame(scaled_features)

for col_name in cols:
    df = df.withColumn(col_name, SF.col(col_name).cast('double'))
    
 

In [0]:
df.show(20)

+---+--------------+----------+--------+-------+----------+--------------------+--------------------+--------------------+------+--------+--------------------+---+-----------+----------------+----------+---------+-------------------+--------------------+-------------------------+----------------------+----------------------------+--------------------+--------------------+-------------------+--------------------+-------------------+-----------------------+--------------+-------------------+-----------------+
|_c0|          CNPJ|  REF_DATE|DATA_REF|CS_ALVO|QTD_SOCIOS|   NATUREZA_JURIDICA|               IDADE|      CAPITAL_SOCIAL|  TIPO|SITUACAO| ATIVIDADE_PRINCIPAL| UF|  MUNICIPIO|    MUNICIPIO_UF|code_muni2|code_muni|       dist_capital|      area_municipio|taxa_atividade_18anosmais|porcent_pop_saneamento|porcent_mulheres10a17_filhos|mortalidade_infantil|                IDHM|         IDHM_renda|  taxa_analfabetismo|     porcent_pobres|municipio_metropolitano|municipio_nome|          pop

In [0]:
df = df.where(SF.col('taxa_analfabetismo') != 'NaN')
df = df.where(SF.col('populacao') != 'NaN')
df.count()

64723

### TIPO [Matriz-Filial]

In [0]:
# importar pacotes
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler

In [0]:
## String Indexer em Tipo
indexer = StringIndexer(inputCol='TIPO', outputCol='tipo_index')

df = indexer.fit(df).transform(df)

## Natureza Juridica

In [0]:
countGenTab(df, 'NATUREZA_JURIDICA').head()

,NATUREZA_JURIDICA,count,prop
0,SOCIEDADE EMPRESARIA LIMITADA,37212,57.49
1,EMPRESARIO (INDIVIDUAL),20784,32.11
2,EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITAD...,6544,10.11
3,SOCIEDADE ANONIMA FECHADA,63,0.10
4,COOPERATIVA,46,0.07


In [0]:
df = df.withColumn('natureza_socied_limitada', when(df.NATUREZA_JURIDICA == 'SOCIEDADE EMPRESARIA LIMITADA', 1).otherwise(0))
df = df.withColumn('natureza_empresario_indiv', when(df.NATUREZA_JURIDICA == 'EMPRESARIO (INDIVIDUAL)', 1).otherwise(0))
df = df.withColumn('natureza_empresa_indiv_limitada', when(df.NATUREZA_JURIDICA == 'EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITADA', 1).otherwise(0))

### SITUAÇÃO


In [0]:
countGenTab(df, 'SITUACAO')

,SITUACAO,count,prop
0,ATIVA,60115,92.88
1,BAIXADA,4093,6.32
2,INAPTA,441,0.68
3,SUSPENSA,74,0.11


In [0]:
df = df.withColumn('situacao_ativa', when(df.SITUACAO == 'ATIVA', 1).otherwise(0))
df = df.withColumn('situacao_baixada', when(df.SITUACAO == 'BAIXADA', 1).otherwise(0))

### UNIDADE FEDERATIVA (UF)

In [0]:
df = df.withColumnRenamed('UF12', 'UF')

prop_UF = countGenTab(df, 'UF')
#prop_UF.head(30)
prop_UF = spark.createDataFrame(prop_UF)
prop_UF = prop_UF.withColumn('SIG', SF.when(SF.col("prop")>1, SF.lit("1")).otherwise(SF.lit('0')))
prop_UF.show(30, False)

+---+-----+-----+---+
|UF |count|prop |SIG|
+---+-----+-----+---+
|SP |25970|40.12|1  |
|RJ |6859 |10.6 |1  |
|MG |6502 |10.05|1  |
|PR |4711 |7.28 |1  |
|RS |4425 |6.84 |1  |
|BA |2796 |4.32 |1  |
|SC |2545 |3.93 |1  |
|PE |1816 |2.81 |1  |
|DF |1364 |2.11 |1  |
|GO |1347 |2.08 |1  |
|ES |1312 |2.03 |1  |
|CE |1147 |1.77 |1  |
|MS |519  |0.8  |0  |
|MT |512  |0.79 |0  |
|PA |444  |0.69 |0  |
|PB |424  |0.66 |0  |
|AM |384  |0.59 |0  |
|MA |339  |0.52 |0  |
|RN |269  |0.42 |0  |
|AL |241  |0.37 |0  |
|SE |214  |0.33 |0  |
|PI |189  |0.29 |0  |
|TO |181  |0.28 |0  |
|RO |76   |0.12 |0  |
|AC |59   |0.09 |0  |
|AP |40   |0.06 |0  |
|RR |38   |0.06 |0  |
+---+-----+-----+---+



In [0]:
prop_UF = prop_UF.drop('count','prop')
df = df.join(prop_UF, ['UF'], 'left')

In [0]:
df = df.withColumn('UF', SF.when(SF.col("SIG")==1, SF.col("UF")).otherwise(SF.lit('OUTROS')))
df = df.drop('SIG')

In [0]:
indexer = StringIndexer(inputCol='UF', outputCol='UF_index')

df = indexer.fit(df).transform(df)

In [0]:
encoder = OneHotEncoderEstimator(inputCols=['UF_index'],
                                 outputCols=['UF_encoded'])

model = encoder.fit(df)
df = model.transform(df)

### ATIVIDADE PRINCIPAL

In [0]:
countGenTab(df, 'ATIVIDADE_PRINCIPAL').head()

,ATIVIDADE_PRINCIPAL,count,prop
0,RESTAURANTES E SIMILARES,17781,27.47
1,"COMERCIO VAREJISTA DE MERCADORIAS EM GERAL, CO...",10667,16.48
2,"LANCHONETES, CASAS DE CHA, DE SUCOS E SIMILARES",10032,15.50
3,"COMERCIO VAREJISTA DE MERCADORIAS EM GERAL, CO...",5937,9.17
4,********,4593,7.10


In [0]:
bag_word = ('DOCES', 'ALIMENTO', 'RESTAURANTE')

@SF.udf('string')
def evalAtividade(value):
  cont = 0
  for word in bag_word:
    if word in value:
      cont = cont + 1
  if cont >= 1:
    return 1
  else:
    return 0

df = df.withColumn('atividade_alimento', evalAtividade('ATIVIDADE_PRINCIPAL'))

In [0]:
countGenTab(df, 'atividade_alimento')

,atividade_alimento,count,prop
0,0,44601,68.91
1,1,20122,31.09


In [0]:
data = df.select('CNPJ', 'DATA_REF', 'CS_ALVO', 'QTD_SOCIOS', 
                 'CAPITAL_SOCIAL', 'IDADE', 'tipo_index', 
                 'natureza_socied_limitada', 'natureza_empresario_indiv',
                 'natureza_empresa_indiv_limitada', 'situacao_ativa',
                'situacao_baixada', 'UF_index', 'atividade_alimento', 'populacao',
                 'municipio_metropolitano','dist_capital','area_municipio',
                 'taxa_atividade_18anosmais','porcent_pop_saneamento',
                 'porcent_mulheres10a17_filhos','mortalidade_infantil','IDHM',
                 'IDHM_renda','taxa_analfabetismo','porcent_pobres')

data.show(5)

+--------------+--------+-------+----------+--------------------+--------------------+----------+------------------------+-------------------------+-------------------------------+--------------+----------------+--------+------------------+-------------------+-----------------------+------------------+--------------------+-------------------------+----------------------+----------------------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|          CNPJ|DATA_REF|CS_ALVO|QTD_SOCIOS|      CAPITAL_SOCIAL|               IDADE|tipo_index|natureza_socied_limitada|natureza_empresario_indiv|natureza_empresa_indiv_limitada|situacao_ativa|situacao_baixada|UF_index|atividade_alimento|          populacao|municipio_metropolitano|      dist_capital|      area_municipio|taxa_atividade_18anosmais|porcent_pop_saneamento|porcent_mulheres10a17_filhos|mortalidade_infantil|               IDHM|         IDHM_renda| taxa_analfabetismo|     porcent_

In [0]:
countGenTab(data, 'CS_ALVO')

,CS_ALVO,count,prop
0,0,57382,88.66
1,1,7341,11.34


In [0]:
# SALVAR BASE FINAL
pdata = data.toPandas()
pdata.to_csv(path+'model_data5.csv')